In [1]:

""" run_mot_challenge.py

Run example:
run_mot_challenge.py --USE_PARALLEL False --METRICS Hota --TRACKERS_TO_EVAL Lif_T

Command Line Arguments: Defaults, # Comments
    Eval arguments:
        'USE_PARALLEL': False,
        'NUM_PARALLEL_CORES': 8,
        'BREAK_ON_ERROR': True,
        'PRINT_RESULTS': True,
        'PRINT_ONLY_COMBINED': False,
        'PRINT_CONFIG': True,
        'TIME_PROGRESS': True,
        'OUTPUT_SUMMARY': True,
        'OUTPUT_DETAILED': True,
        'PLOT_CURVES': True,
    Dataset arguments:
        'GT_FOLDER': os.path.join(code_path, 'data/gt/mot_challenge/'),  # Location of GT data
        'TRACKERS_FOLDER': os.path.join(code_path, 'data/trackers/mot_challenge/'),  # Trackers location
        'OUTPUT_FOLDER': None,  # Where to save eval results (if None, same as TRACKERS_FOLDER)
        'TRACKERS_TO_EVAL': None,  # Filenames of trackers to eval (if None, all in folder)
        'CLASSES_TO_EVAL': ['pedestrian'],  # Valid: ['pedestrian']
        'BENCHMARK': 'MOT17',  # Valid: 'MOT17', 'MOT16', 'MOT20', 'MOT15'
        'SPLIT_TO_EVAL': 'train',  # Valid: 'train', 'test', 'all'
        'INPUT_AS_ZIP': False,  # Whether tracker input files are zipped
        'PRINT_CONFIG': True,  # Whether to print current config
        'DO_PREPROC': True,  # Whether to perform preprocessing (never done for 2D_MOT_2015)
        'TRACKER_SUB_FOLDER': 'data',  # Tracker files are in TRACKER_FOLDER/tracker_name/TRACKER_SUB_FOLDER
        'OUTPUT_SUB_FOLDER': '',  # Output files are saved in OUTPUT_FOLDER/tracker_name/OUTPUT_SUB_FOLDER
    Metric arguments:
        'METRICS': ['HOTA', 'CLEAR', 'Identity', 'VACE']
"""

import sys
import os
import argparse
from multiprocessing import freeze_support
import numpy as np
import random

sys.path.append('../')
import trackeval  # noqa: E402


In [3]:
# Command line interface:
default_eval_config = trackeval.Evaluator.get_default_eval_config()
default_eval_config['DISPLAY_LESS_PROGRESS'] = False
default_dataset_config = trackeval.datasets.MotChallenge2DBox.get_default_dataset_config()
default_metrics_config = {'METRICS': ['HOTA', 'CLEAR', 'Identity'], 'THRESHOLD': 0.5}
config = {**default_eval_config, **default_dataset_config, **default_metrics_config}  # Merge default configs
parser = argparse.ArgumentParser()
for setting in config.keys():
    if type(config[setting]) == list or type(config[setting]) == type(None):
        parser.add_argument("--" + setting, nargs='+')
    else:
        parser.add_argument("--" + setting)
args = parser.parse_args(args=[]).__dict__
for setting in args.keys():
    if args[setting] is not None:
        if type(config[setting]) == type(True):
            if args[setting] == 'True':
                x = True
            elif args[setting] == 'False':
                x = False
            else:
                raise Exception('Command line parameter ' + setting + 'must be True or False')
        elif type(config[setting]) == type(1):
            x = int(args[setting])
        elif type(args[setting]) == type(None):
            x = None
        elif setting == 'SEQ_INFO':
            x = dict(zip(args[setting], [None]*len(args[setting])))
        else:
            x = args[setting]
        config[setting] = x
eval_config = {k: v for k, v in config.items() if k in default_eval_config.keys()}
dataset_config = {k: v for k, v in config.items() if k in default_dataset_config.keys()}
metrics_config = {k: v for k, v in config.items() if k in default_metrics_config.keys()}


In [10]:
# Run code
evaluator = trackeval.Evaluator(eval_config)
dataset_list = [trackeval.datasets.MotChallenge2DBox(dataset_config)]
metrics_list = []
for metric in [trackeval.metrics.HOTA, trackeval.metrics.CLEAR, trackeval.metrics.Identity, trackeval.metrics.VACE]:
    if metric.get_name() in metrics_config['METRICS']:
        metrics_list.append(metric(metrics_config))
if len(metrics_list) == 0:
    raise Exception('No metrics selected for evaluation')
# evaluator.evaluate(dataset_list, metrics_list)


Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/guest/dev_repo/MOT_eval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /home/guest/dev_repo/MOT_eval/data/gt/mot_challenge/
TRACKERS_FOLDER      : /home/guest/dev_repo/MOT_eval/data/t

In [15]:
dataset_list[0].__dict__

{'tracker_list': ['MPNTrack'],
 'seq_list': ['MOT17-02-DPM',
  'MOT17-04-DPM',
  'MOT17-05-DPM',
  'MOT17-09-DPM',
  'MOT17-10-DPM',
  'MOT17-11-DPM',
  'MOT17-13-DPM',
  'MOT17-02-FRCNN',
  'MOT17-04-FRCNN',
  'MOT17-05-FRCNN',
  'MOT17-09-FRCNN',
  'MOT17-10-FRCNN',
  'MOT17-11-FRCNN',
  'MOT17-13-FRCNN',
  'MOT17-02-SDP',
  'MOT17-04-SDP',
  'MOT17-05-SDP',
  'MOT17-09-SDP',
  'MOT17-10-SDP',
  'MOT17-11-SDP',
  'MOT17-13-SDP'],
 'class_list': ['pedestrian'],
 'output_fol': '/home/guest/dev_repo/MOT_eval/data/trackers/mot_challenge/MOT17-train',
 'output_sub_fol': '',
 'should_classes_combine': False,
 'use_super_categories': False,
 'config': {'PRINT_CONFIG': True,
  'GT_FOLDER': '/home/guest/dev_repo/MOT_eval/data/gt/mot_challenge/',
  'TRACKERS_FOLDER': '/home/guest/dev_repo/MOT_eval/data/trackers/mot_challenge/',
  'OUTPUT_FOLDER': None,
  'TRACKERS_TO_EVAL': None,
  'CLASSES_TO_EVAL': ['pedestrian'],
  'BENCHMARK': 'MOT17',
  'SPLIT_TO_EVAL': 'train',
  'INPUT_AS_ZIP': False,
 

In [11]:
import time
import traceback
from multiprocessing.pool import Pool
from functools import partial
import os
from trackeval import utils
from trackeval.utils import TrackEvalException
from trackeval import _timing
from trackeval.metrics import Count

def evaluate(dataset_list, metrics_list):
    """Evaluate a set of metrics on a set of datasets"""
    # config = config
    metrics_list = metrics_list + [Count()]  # Count metrics are always run
    metric_names = utils.validate_metrics_list(metrics_list)
    dataset_names = [dataset.get_name() for dataset in dataset_list]
    print(dataset_names)
    output_res = {}
    output_msg = {}

    for dataset, dataset_name in zip(dataset_list, dataset_names):
        # Get dataset info about what to evaluate
        output_res[dataset_name] = {}
        print('datast',dataset)
        output_msg[dataset_name] = {}
        tracker_list, seq_list, class_list = dataset.get_eval_info()
        print(tracker_list)
        print('\nEvaluating %i tracker(s) on %i sequence(s) for %i class(es) on %s dataset using the following '
                'metrics: %s\n' % (len(tracker_list), len(seq_list), len(class_list), dataset_name,
                                    ', '.join(metric_names)))
        
        # Evaluate each tracker
        for tracker in tracker_list:
        # if not config['BREAK_ON_ERROR'] then go to next tracker without breaking
            try:
                # Evaluate each sequence in parallel or in series.
                # returns a nested dict (res), indexed like: res[seq][class][metric_name][sub_metric field]
                # e.g. res[seq_0001][pedestrian][hota][DetA]
                print('\nEvaluating %s\n' % tracker)
                time_start = time.time()
                if config['USE_PARALLEL']:
                    with Pool(config['NUM_PARALLEL_CORES']) as pool:
                        _eval_sequence = partial(eval_sequence, dataset=dataset, tracker=tracker,
                                                    class_list=class_list, metrics_list=metrics_list,
                                                    metric_names=metric_names)
                        results = pool.map(_eval_sequence, seq_list)
                        res = dict(zip(seq_list, results))

                else:
                    res = {}
                    for curr_seq in sorted(seq_list):
                        res[curr_seq] = eval_sequence(curr_seq, dataset, tracker, class_list, metrics_list,
                                                        metric_names)     
                        # print(res) 
                    return res                                                                    
                
            except Exception as err:
                continue

# @_timing.time
def eval_sequence(seq, dataset, tracker, class_list, metrics_list, metric_names):
    """Function for evaluating a single sequence"""

    raw_data = dataset.get_raw_seq_data(tracker, seq) ##ここにbbox_dfとかが入ってほしい。
    seq_res = {}
    # print(class_list)
    for cls in class_list:
        seq_res[cls] = {}
        # print(seq_res)
        data = dataset.get_preprocessed_seq_data(raw_data, cls)
        # print(data.keys())
        for metric, met_name in zip(metrics_list, metric_names):
            seq_res[cls][met_name] = metric.eval_sequence(data)
    return seq_res


In [12]:
evaluate(dataset_list, metrics_list)

['MotChallenge2DBox']
datast <trackeval.datasets.mot_challenge_2d_box.MotChallenge2DBox object at 0x7f0e5764ef40>
['MPNTrack']

Evaluating 1 tracker(s) on 21 sequence(s) for 1 class(es) on MotChallenge2DBox dataset using the following metrics: HOTA, CLEAR, Identity, Count


Evaluating MPNTrack

    MotChallenge2DBox.get_raw_seq_data(MPNTrack, MOT17-02-DPM)             0.2384 sec
    MotChallenge2DBox.get_preprocessed_seq_data(pedestrian)                0.2103 sec
    HOTA.eval_sequence()                                                   0.2276 sec
    CLEAR.eval_sequence()                                                  0.0401 sec
    Identity.eval_sequence()                                               0.0092 sec
    Count.eval_sequence()                                                  0.0000 sec
    MotChallenge2DBox.get_raw_seq_data(MPNTrack, MOT17-02-FRCNN)           0.2166 sec
    MotChallenge2DBox.get_preprocessed_seq_data(pedestrian)                0.1889 sec
    HOTA.eval_se

KeyboardInterrupt: 